<a href="https://colab.research.google.com/github/ElenaNtm/IPTO-Data-projections/blob/main/ipto_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [ ]:
import pandas as pd
#import requests
from datetime import datetime, timedelta
#from datetime import datetime, timedelta
from urllib.error import HTTPError
#import re
#from urllib.parse import urlparse

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


#Preliminaries

In [ ]:
# Initialize an empty DataFrame
#df = pd.DataFrame()
#df_not_found = pd.DataFrame()
df_hydro = pd.DataFrame()
df_lignite = pd.DataFrame()
df_net_load = pd.DataFrame()
df_gas = pd.DataFrame()
df_res = pd.DataFrame()
df_inter = pd.DataFrame()

In [ ]:

"""
Οι ημ/νιες από 01/08/2020 δεν έχουν κάποιο θέμα εκτός από τη τελευταία μέρα του μήνα το οποίο θα το λύσουμε
θα το σπάσουμε ανά χρόνο γιατί κρασάρει η διαδικασία αλλιώς
"""
base_url = "https://www.admie.gr/sites/default/files/attached-files/type-file/"

#From May 2020


In [ ]:
#Manualy change the dates because if loaded all the system will crush
start_date = datetime(2020, 6, 1)
end_date = datetime(2020, 12, 31)
date_format = "%Y/%m/%Y%m%d"
current_date = start_date

In [ ]:
while current_date <= end_date:
    date_str = current_date.strftime(date_format)
    url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

    try:
        # Read the XLSX file into a pandas DataFrame
        df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
        #Hydro
        substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL HYDRO'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
        df_hydro = pd.concat([df_hydro, row])
        df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Lignite
        substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL LIGNITE'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

        df_lignite = pd.concat([df_lignite, row])
        df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Gas
        substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL GAS'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

        df_gas = pd.concat([df_gas, row])
        df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #RES
        substring = 'TOTAL RES'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

        df_res = pd.concat([df_res, row])
        df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Interconections
        substring = 'EXPORTS-IMPORTS'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

        df_inter = pd.concat([df_inter, row])
        df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Net Load
        substring = 'NET LOAD'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

        df_net_load = pd.concat([df_net_load, row])
        df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
    except HTTPError as e:
        print(f"Error accessing URL: {url}")
        #print(e)


    # Increment to the next date
    current_date += timedelta(days=1)

Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/06/20200630_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/07/20200731_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/08/20200831_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/09/20200930_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/10/20201031_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/11/20201130_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/12/20201231_SystemRealizationSCADA_01.xls


In [ ]:
"""
2020 Not found urls
"""
url6 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/07/20200630_SystemRealizationSCADA_01.xls'
url7 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/08/20200731_SystemRealizationSCADA_01.xls'
url8 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/09/20200831_SystemRealizationSCADA_01.xls'
url9 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/10/20200930_SystemRealizationSCADA_01.xls'
url10 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/11/20201031_SystemRealizationSCADA_01.xls'
url11 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/12/20201130_SystemRealizationSCADA_01.xls'
url12 = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2021/01/20201231_SystemRealizationSCADA_01.xls'
url = [url6,url7,url8,url9,url10,url11,url12]
date = {url6: '2020/07/20200630', #June
        url7: '2020/07/20200731', #July
        url8: '2020/07/20200831', #August
        url9: '2020/07/20200930', #September
        url10: '2020/07/20201031', #Octomber
        url11: '2020/07/20201130', #November
        url12: '2020/07/20201031', #December
}

In [ ]:
for i in url:
    df2 = pd.read_excel(i, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
    #Hydro
    substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL HYDRO'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date[i]])
    df_hydro = pd.concat([df_hydro, row])
    df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Lignite
    substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL LIGNITE'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date[i]])

    df_lignite = pd.concat([df_lignite, row])
    df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Gas
    substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL GAS'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date[i]])

    df_gas = pd.concat([df_gas, row])
    df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #RES
    substring = 'TOTAL RES'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total RES'], index=[date[i]])

    df_res = pd.concat([df_res, row])
    df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Interconections
    substring = 'EXPORTS-IMPORTS'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date[i]])

    df_inter = pd.concat([df_inter, row])
    df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date[i]})
    #Net Load
    substring = 'NET LOAD'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date[i]])

    df_net_load = pd.concat([df_net_load, row])
    df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date[i]})


##Save the data for 2020

##Drop duplicates


In [ ]:
#RES
df_res['index'] = df_res.index
df_res = df_res.drop_duplicates(subset='index')
df_res.drop(['index'], axis = 1, inplace = True)

#HYDRO
df_hydro['index'] = df_hydro.index
df_hydro = df_hydro.drop_duplicates(subset='index')
df_hydro.drop(['index'], axis = 1, inplace = True)

#LIGNITE
df_lignite['index'] = df_lignite.index
df_lignite = df_lignite.drop_duplicates(subset='index')
df_lignite.drop(['index'], axis = 1, inplace = True)

#NET LOAD
df_net_load['index'] = df_net_load.index
df_net_load = df_net_load.drop_duplicates(subset='index')
df_net_load.drop(['index'], axis = 1, inplace = True)

#GAS
df_gas['index'] = df_gas.index
df_gas = df_gas.drop_duplicates(subset='index')
df_gas.drop(['index'], axis = 1, inplace = True)

#GAS
df_inter['index'] = df_inter.index
df_inter = df_inter.drop_duplicates(subset='index')
df_inter.drop(['index'], axis = 1, inplace = True)

<ipython-input-9-5932f93a05b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res.drop(['index'], axis = 1, inplace = True)
<ipython-input-9-5932f93a05b3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hydro.drop(['index'], axis = 1, inplace = True)
<ipython-input-9-5932f93a05b3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lignite.drop(['index'], axis = 1, inplace = True)
<ipython-input-9-5932f93a05b3>:24: SettingWit

##Indexes

In [ ]:
df_gas.index = df_gas.index.str[8:]
df_hydro.index = df_hydro.index.str[8:]
df_lignite.index = df_lignite.index.str[8:]
df_net_load.index = df_net_load.index.str[8:]
df_res.index = df_res.index.str[8:]
df_inter.index = df_inter.index.str[8:]

##Datetime format

In [ ]:
df_res.index = pd.to_datetime(df_res.index, format='%Y/%m/%d')
df_lignite.index = pd.to_datetime(df_lignite.index, format='%Y/%m/%d')
df_net_load.index = pd.to_datetime(df_net_load.index, format='%Y/%m/%d')
df_hydro.index = pd.to_datetime(df_hydro.index, format='%Y/%m/%d')
df_gas.index = pd.to_datetime(df_gas.index, format='%Y/%m/%d')
df_inter.index = pd.to_datetime(df_inter.index, format= '%Y/%m/%d')

##Save to drive


In [ ]:
df_gas.to_csv('gas_2020.csv')
!cp gas_2020.csv "drive/My Drive/"

In [ ]:
df_res.to_csv('res_2020.csv')
!cp res_2020.csv "drive/My Drive/"

In [ ]:
df_lignite.to_csv('lignite_2020.csv')
!cp lignite_2020.csv "drive/My Drive/"

In [ ]:
df_net_load.to_csv('net_load_2020.csv')
!cp net_load_2020.csv "drive/My Drive/"

In [ ]:
df_hydro.to_csv('hydro_2020.csv')
!cp hydro_2020.csv "drive/My Drive/"

In [ ]:
df_inter.to_csv('inter_2020.csv')
!cp inter_2020.csv "drive/My Drive/"

#Preliminaries

In [ ]:
# Initialize an empty DataFrame
#df = pd.DataFrame()
#df_not_found = pd.DataFrame()
df_hydro = pd.DataFrame()
df_lignite = pd.DataFrame()
df_net_load = pd.DataFrame()
df_gas = pd.DataFrame()
df_res = pd.DataFrame()
df_inter = pd.DataFrame()

#2021

In [ ]:
#Manualy change the dates because if loaded all the system will crush
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 12, 31)
date_format = "%Y/%m/%Y%m%d"
current_date = start_date

In [ ]:
while current_date <= end_date:
    date_str = current_date.strftime(date_format)
    url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

    try:
        # Read the XLSX file into a pandas DataFrame
        df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
        #Hydro
        substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL HYDRO'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
        df_hydro = pd.concat([df_hydro, row])
        df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Lignite
        substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL LIGNITE'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

        df_lignite = pd.concat([df_lignite, row])
        df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Gas
        substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL GAS'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

        df_gas = pd.concat([df_gas, row])
        df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #RES
        substring = 'TOTAL RES'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

        df_res = pd.concat([df_res, row])
        df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Interconections
        substring = 'EXPORTS-IMPORTS'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

        df_inter = pd.concat([df_inter, row])
        df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Net Load
        substring = 'NET LOAD'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

        df_net_load = pd.concat([df_net_load, row])
        df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
    except HTTPError as e:
        print(f"Error accessing URL: {url}")
        #print(e)


    # Increment to the next date
    current_date += timedelta(days=1)

Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/01/20210131_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/02/20210228_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/03/20210331_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/04/20210430_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/05/20210531_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/06/20210630_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2021/07/20210731_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/

In [ ]:
df_inter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, 2021/01/20210101 to 2021/12/20211230
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Exports - Imports  353 non-null    int64
dtypes: int64(1)
memory usage: 5.5+ KB


In [ ]:
url1 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/02/20210131_SystemRealizationSCADA_01.xls"
url2 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/03/20210228_SystemRealizationSCADA_01.xls"
url3 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/04/20210331_SystemRealizationSCADA_01.xls"
url4 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/05/20210430_SystemRealizationSCADA_01.xls"
url5 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/06/20210531_SystemRealizationSCADA_01.xls"
url6 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/07/20210630_SystemRealizationSCADA_01.xls"
url7 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/08/20210731_SystemRealizationSCADA_01.xls"
url8 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/09/20210831_SystemRealizationSCADA_01.xls"
url9 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/10/20210930_SystemRealizationSCADA_01.xls"
url10 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/11/20211031_SystemRealizationSCADA_01.xls"
url11 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2021/12/20211130_SystemRealizationSCADA_01.xls"
url12 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/01/20211231_SystemRealizationSCADA_01.xls"

url = [url1, url2, url3, url4, url5, url6, url7, url8, url9, url10, url11, url12]
date = {url1: '2020/07/20210131', #June
        url2: '2020/07/20210228', #July
        url3: '2020/07/20210331', #August
        url4: '2020/07/20210430', #September
        url5: '2020/07/20210531', #Octomber
        url6: '2020/07/20210630', #June
        url7: '2020/07/20210731', #July
        url8: '2020/07/20210831', #August
        url9: '2020/07/20210930', #September
        url10: '2020/07/20211031', #Octomber
        url11: '2020/07/20211130', #November
        url12: '2020/07/20211031' #December
}

In [ ]:
for i in url:
    df2 = pd.read_excel(i, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
    #Hydro
    substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL HYDRO'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date[i]])
    df_hydro = pd.concat([df_hydro, row])
    df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Lignite
    substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL LIGNITE'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date[i]])

    df_lignite = pd.concat([df_lignite, row])
    df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Gas
    substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL GAS'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date[i]])

    df_gas = pd.concat([df_gas, row])
    df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #RES
    substring = 'TOTAL RES'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total RES'], index=[date[i]])

    df_res = pd.concat([df_res, row])
    df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Interconections
    substring = 'EXPORTS-IMPORTS'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date[i]])

    df_inter = pd.concat([df_inter, row])
    df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date[i]})
    #Net Load
    substring = 'NET LOAD'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date[i]])

    df_net_load = pd.concat([df_net_load, row])
    df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date[i]})


##Save the data for 2021



##Drop duplicates


In [ ]:
#RES
df_res['index'] = df_res.index
df_res = df_res.drop_duplicates(subset='index')
df_res.drop(['index'], axis = 1, inplace = True)

#HYDRO
df_hydro['index'] = df_hydro.index
df_hydro = df_hydro.drop_duplicates(subset='index')
df_hydro.drop(['index'], axis = 1, inplace = True)

#LIGNITE
df_lignite['index'] = df_lignite.index
df_lignite = df_lignite.drop_duplicates(subset='index')
df_lignite.drop(['index'], axis = 1, inplace = True)

#NET LOAD
df_net_load['index'] = df_net_load.index
df_net_load = df_net_load.drop_duplicates(subset='index')
df_net_load.drop(['index'], axis = 1, inplace = True)

#GAS
df_gas['index'] = df_gas.index
df_gas = df_gas.drop_duplicates(subset='index')
df_gas.drop(['index'], axis = 1, inplace = True)

#GAS
df_inter['index'] = df_inter.index
df_inter = df_inter.drop_duplicates(subset='index')
df_inter.drop(['index'], axis = 1, inplace = True)

<ipython-input-24-5932f93a05b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res.drop(['index'], axis = 1, inplace = True)
<ipython-input-24-5932f93a05b3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hydro.drop(['index'], axis = 1, inplace = True)
<ipython-input-24-5932f93a05b3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lignite.drop(['index'], axis = 1, inplace = True)
<ipython-input-24-5932f93a05b3>:24: Settin

##Indexes

In [ ]:
df_gas.index = df_gas.index.str[8:]
df_hydro.index = df_hydro.index.str[8:]
df_lignite.index = df_lignite.index.str[8:]
df_net_load.index = df_net_load.index.str[8:]
df_res.index = df_res.index.str[8:]
df_inter.index = df_inter.index.str[8:]

##Datetime format

In [ ]:
df_res.index = pd.to_datetime(df_res.index, format='%Y/%m/%d')
df_lignite.index = pd.to_datetime(df_lignite.index, format='%Y/%m/%d')
df_net_load.index = pd.to_datetime(df_net_load.index, format='%Y/%m/%d')
df_hydro.index = pd.to_datetime(df_hydro.index, format='%Y/%m/%d')
df_gas.index = pd.to_datetime(df_gas.index, format='%Y/%m/%d')
df_inter.index = pd.to_datetime(df_inter.index, format= '%Y/%m/%d')

##Save to drive


In [ ]:
df_gas.to_csv('gas_2021.csv')
!cp gas_2021.csv "drive/My Drive/"

In [ ]:
df_res.to_csv('res_2021.csv')
!cp res_2021.csv "drive/My Drive/"

In [ ]:
df_lignite.to_csv('lignite_2021.csv')
!cp lignite_2021.csv "drive/My Drive/"

In [ ]:
df_net_load.to_csv('net_load_2021.csv')
!cp net_load_2021.csv "drive/My Drive/"

In [ ]:
df_hydro.to_csv('hydro_2021.csv')
!cp hydro_2021.csv "drive/My Drive/"

In [ ]:
df_inter.to_csv('inter_2021.csv')
!cp inter_2021.csv "drive/My Drive/"

#Preliminaries

In [ ]:
# Initialize an empty DataFrame
#df = pd.DataFrame()
#df_not_found = pd.DataFrame()
df_hydro = pd.DataFrame()
df_lignite = pd.DataFrame()
df_net_load = pd.DataFrame()
df_gas = pd.DataFrame()
df_res = pd.DataFrame()
df_inter = pd.DataFrame()

#2022

In [ ]:
#Manualy change the dates because if loaded all the system will crush
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)
date_format = "%Y/%m/%Y%m%d"
current_date = start_date

In [ ]:
while current_date <= end_date:
    date_str = current_date.strftime(date_format)
    url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

    try:
        # Read the XLSX file into a pandas DataFrame
        df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
        #Hydro
        substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL HYDRO'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
        df_hydro = pd.concat([df_hydro, row])
        df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Lignite
        substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL LIGNITE'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

        df_lignite = pd.concat([df_lignite, row])
        df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Gas
        substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL GAS'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

        df_gas = pd.concat([df_gas, row])
        df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #RES
        substring = 'TOTAL RES'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

        df_res = pd.concat([df_res, row])
        df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Interconections
        substring = 'EXPORTS-IMPORTS'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

        df_inter = pd.concat([df_inter, row])
        df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Net Load
        substring = 'NET LOAD'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

        df_net_load = pd.concat([df_net_load, row])
        df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
    except HTTPError as e:
        print(f"Error accessing URL: {url}")
        #print(e)


    # Increment to the next date
    current_date += timedelta(days=1)

Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/01/20220131_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/02/20220228_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/03/20220331_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/04/20220430_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/05/20220531_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/06/20220630_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2022/07/20220731_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/

In [ ]:

url1 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/02/20220131_SystemRealizationSCADA_01.xls"
url2 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/03/20220228_SystemRealizationSCADA_01.xls"
url3 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/04/20220331_SystemRealizationSCADA_01.xls"
url4 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/05/20220430_SystemRealizationSCADA_01.xls"
url5 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/06/20220531_SystemRealizationSCADA_01.xls"
url6 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/07/20220630_SystemRealizationSCADA_01.xls"
url7 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/08/20220731_SystemRealizationSCADA_01.xls"
url8 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/09/20220831_SystemRealizationSCADA_01.xls"
url9 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/10/20220930_SystemRealizationSCADA_01.xls"
url10 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/11/20221031_SystemRealizationSCADA_01.xls"
url11 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2022/12/20221130_SystemRealizationSCADA_01.xls"
url12 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/01/20221231_SystemRealizationSCADA_01.xls"
url = [url1, url2, url3, url4, url5, url6, url7, url8, url9, url10, url11, url12]
date = {url1: '2020/07/20220131', #June
        url2: '2020/07/20220228', #July
        url3: '2020/07/20220331', #August
        url4: '2020/07/20220430', #September
        url5: '2020/07/20220531', #Octomber
        url6: '2020/07/20220630', #June
        url7: '2020/07/20220731', #July
        url8: '2020/07/20220831', #August
        url9: '2020/07/20220930', #September
        url10: '2020/07/20221031', #Octomber
        url11: '2020/07/20221130', #November
        url12: '2020/07/20221031' #December
}

In [ ]:
for i in url:
    df2 = pd.read_excel(i, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
    #Hydro
    substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL HYDRO'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date[i]])
    df_hydro = pd.concat([df_hydro, row])
    df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Lignite
    substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL LIGNITE'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date[i]])

    df_lignite = pd.concat([df_lignite, row])
    df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Gas
    substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL GAS'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date[i]])

    df_gas = pd.concat([df_gas, row])
    df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #RES
    substring = 'TOTAL RES'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total RES'], index=[date[i]])

    df_res = pd.concat([df_res, row])
    df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Interconections
    substring = 'EXPORTS-IMPORTS'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date[i]])

    df_inter = pd.concat([df_inter, row])
    df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date[i]})
    #Net Load
    substring = 'NET LOAD'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date[i]])

    df_net_load = pd.concat([df_net_load, row])
    df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date[i]})


##Drop duplicates


In [ ]:
#RES
df_res['index'] = df_res.index
df_res = df_res.drop_duplicates(subset='index')
df_res.drop(['index'], axis = 1, inplace = True)

#HYDRO
df_hydro['index'] = df_hydro.index
df_hydro = df_hydro.drop_duplicates(subset='index')
df_hydro.drop(['index'], axis = 1, inplace = True)

#LIGNITE
df_lignite['index'] = df_lignite.index
df_lignite = df_lignite.drop_duplicates(subset='index')
df_lignite.drop(['index'], axis = 1, inplace = True)

#NET LOAD
df_net_load['index'] = df_net_load.index
df_net_load = df_net_load.drop_duplicates(subset='index')
df_net_load.drop(['index'], axis = 1, inplace = True)

#GAS
df_gas['index'] = df_gas.index
df_gas = df_gas.drop_duplicates(subset='index')
df_gas.drop(['index'], axis = 1, inplace = True)

#GAS
df_inter['index'] = df_inter.index
df_inter = df_inter.drop_duplicates(subset='index')
df_inter.drop(['index'], axis = 1, inplace = True)

<ipython-input-38-5932f93a05b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res.drop(['index'], axis = 1, inplace = True)
<ipython-input-38-5932f93a05b3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hydro.drop(['index'], axis = 1, inplace = True)
<ipython-input-38-5932f93a05b3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lignite.drop(['index'], axis = 1, inplace = True)
<ipython-input-38-5932f93a05b3>:24: Settin

##Indexes

In [ ]:
df_gas.index = df_gas.index.str[8:]
df_hydro.index = df_hydro.index.str[8:]
df_lignite.index = df_lignite.index.str[8:]
df_net_load.index = df_net_load.index.str[8:]
df_res.index = df_res.index.str[8:]
df_inter.index = df_inter.index.str[8:]

##Datetime format

In [ ]:
df_res.index = pd.to_datetime(df_res.index, format='%Y/%m/%d')
df_lignite.index = pd.to_datetime(df_lignite.index, format='%Y/%m/%d')
df_net_load.index = pd.to_datetime(df_net_load.index, format='%Y/%m/%d')
df_hydro.index = pd.to_datetime(df_hydro.index, format='%Y/%m/%d')
df_gas.index = pd.to_datetime(df_gas.index, format='%Y/%m/%d')
df_inter.index = pd.to_datetime(df_inter.index, format= '%Y/%m/%d')

##Save to drive


In [ ]:
df_gas.to_csv('gas_2022.csv')
!cp gas_2022.csv "drive/My Drive/"

In [ ]:
df_res.to_csv('res_2022.csv')
!cp res_2022.csv "drive/My Drive/"

In [ ]:
df_lignite.to_csv('lignite_2022.csv')
!cp lignite_2022.csv "drive/My Drive/"

In [ ]:
df_net_load.to_csv('net_load_2022.csv')
!cp net_load_2022.csv "drive/My Drive/"

In [ ]:
df_hydro.to_csv('hydro_2022.csv')
!cp hydro_2022.csv "drive/My Drive/"

In [ ]:
df_inter.to_csv('inter_2022.csv')
!cp inter_2022.csv "drive/My Drive/"

#Preliminaries

In [ ]:
# Initialize an empty DataFrame
#df = pd.DataFrame()
#df_not_found = pd.DataFrame()
df_hydro = pd.DataFrame()
df_lignite = pd.DataFrame()
df_net_load = pd.DataFrame()
df_gas = pd.DataFrame()
df_res = pd.DataFrame()
df_inter = pd.DataFrame()

#2023

In [ ]:
#Manualy change the dates because if loaded all the system will crush
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 11, 1)
date_format = "%Y/%m/%Y%m%d"
current_date = start_date

In [ ]:
while current_date <= end_date:
    date_str = current_date.strftime(date_format)
    url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

    try:
        # Read the XLSX file into a pandas DataFrame
        df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
        #Hydro
        substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL HYDRO'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
        df_hydro = pd.concat([df_hydro, row])
        df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Lignite
        substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL LIGNITE'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

        df_lignite = pd.concat([df_lignite, row])
        df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Gas
        substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL GAS'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

        df_gas = pd.concat([df_gas, row])
        df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #RES
        substring = 'TOTAL RES'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

        df_res = pd.concat([df_res, row])
        df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Interconections
        substring = 'EXPORTS-IMPORTS'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

        df_inter = pd.concat([df_inter, row])
        df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #Net Load
        substring = 'NET LOAD'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

        df_net_load = pd.concat([df_net_load, row])
        df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
    except HTTPError as e:
        print(f"Error accessing URL: {url}")
        #print(e)


    # Increment to the next date
    current_date += timedelta(days=1)

Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/01/20230131_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/02/20230228_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/03/20230331_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/04/20230430_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/05/20230531_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/06/20230630_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2023/07/20230731_SystemRealizationSCADA_01.xls
Error accessing URL: https://www.admie.gr/sites/default/files/

In [ ]:
url1 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/02/20230131_SystemRealizationSCADA_01.xls"
url2 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/03/20230228_SystemRealizationSCADA_01.xls"
url3 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/04/20230331_SystemRealizationSCADA_01.xls"
url4 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/05/20230430_SystemRealizationSCADA_01.xls"
url5 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/06/20230531_SystemRealizationSCADA_01.xls"
url6 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/07/20230630_SystemRealizationSCADA_01.xls"
url7 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/08/20230731_SystemRealizationSCADA_01.xls"
url8 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/09/20230831_SystemRealizationSCADA_01.xls"
url9 = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/10/20230930_SystemRealizationSCADA_01.xls"
url = [url1, url2, url3, url4, url5, url6, url7, url8, url9]
date = {url1: '2020/07/20230131', #June
        url2: '2020/07/20230228', #July
        url3: '2020/07/20230331', #August
        url4: '2020/07/20230430', #September
        url5: '2020/07/20230531', #Octomber
        url6: '2020/07/20230630', #June
        url7: '2020/07/20230731', #July
        url8: '2020/07/20230831', #August
        url9: '2020/07/20230930', #August
        }

In [ ]:
for i in url:
    df2 = pd.read_excel(i, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
    #Hydro
    substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL HYDRO'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date[i]])
    df_hydro = pd.concat([df_hydro, row])
    df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Lignite
    substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL LIGNITE'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date[i]])

    df_lignite = pd.concat([df_lignite, row])
    df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Gas
    substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'TOTAL GAS'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date[i]])

    df_gas = pd.concat([df_gas, row])
    df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #RES
    substring = 'TOTAL RES'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total RES'], index=[date[i]])

    df_res = pd.concat([df_res, row])
    df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date[i]})

    #Interconections
    substring = 'EXPORTS-IMPORTS'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date[i]])

    df_inter = pd.concat([df_inter, row])
    df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date[i]})
    #Net Load
    substring = 'NET LOAD'
    filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
    wanted_index = filtered.index.tolist()

    if not wanted_index:
        substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
        filtered = df2[df2.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

    value = df2.iloc[wanted_index[len(wanted_index)-1],26]
    row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date[i]])

    df_net_load = pd.concat([df_net_load, row])
    df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date[i]})


##Drop duplicates


In [ ]:
#RES
df_res['index'] = df_res.index
df_res = df_res.drop_duplicates(subset='index')
df_res.drop(['index'], axis = 1, inplace = True)

#HYDRO
df_hydro['index'] = df_hydro.index
df_hydro = df_hydro.drop_duplicates(subset='index')
df_hydro.drop(['index'], axis = 1, inplace = True)

#LIGNITE
df_lignite['index'] = df_lignite.index
df_lignite = df_lignite.drop_duplicates(subset='index')
df_lignite.drop(['index'], axis = 1, inplace = True)

#NET LOAD
df_net_load['index'] = df_net_load.index
df_net_load = df_net_load.drop_duplicates(subset='index')
df_net_load.drop(['index'], axis = 1, inplace = True)

#GAS
df_gas['index'] = df_gas.index
df_gas = df_gas.drop_duplicates(subset='index')
df_gas.drop(['index'], axis = 1, inplace = True)

#GAS
df_inter['index'] = df_inter.index
df_inter = df_inter.drop_duplicates(subset='index')
df_inter.drop(['index'], axis = 1, inplace = True)

##Indexes

In [ ]:
df_gas.index = df_gas.index.str[8:]
df_hydro.index = df_hydro.index.str[8:]
df_lignite.index = df_lignite.index.str[8:]
df_net_load.index = df_net_load.index.str[8:]
df_res.index = df_res.index.str[8:]
df_inter.index = df_inter.index.str[8:]

AttributeError: ignored

##Datetime format

In [ ]:
df_res.index = pd.to_datetime(df_res.index, format='%Y/%m/%d')
df_lignite.index = pd.to_datetime(df_lignite.index, format='%Y/%m/%d')
df_net_load.index = pd.to_datetime(df_net_load.index, format='%Y/%m/%d')
df_hydro.index = pd.to_datetime(df_hydro.index, format='%Y/%m/%d')
df_gas.index = pd.to_datetime(df_gas.index, format='%Y/%m/%d')
df_inter.index = pd.to_datetime(df_inter.index, format= '%Y/%m/%d')

##Save to drive


In [ ]:
df_gas.to_csv('gas_2023.csv')
!cp gas_2023.csv "drive/My Drive/"

In [ ]:
df_res.to_csv('res_2023.csv')
!cp res_2023.csv "drive/My Drive/"

In [ ]:
df_lignite.to_csv('lignite_2023.csv')
!cp lignite_2023.csv "drive/My Drive/"

In [ ]:
df_net_load.to_csv('net_load_2023.csv')
!cp net_load_2023.csv "drive/My Drive/"

In [ ]:
df_hydro.to_csv('hydro_2023.csv')
!cp hydro_2023.csv "drive/My Drive/"

In [ ]:
df_inter.to_csv('inter_2023.csv')
!cp inter_2023.csv "drive/My Drive/"